In [1]:
import sympy as sp
from sympy import Matrix, BlockMatrix, MatrixSymbol

import sympy_matrix_utils as sp_utils 

## Testing the methods

In [2]:
y_1 = sp_utils.create_block_vector('y_1', V_fix_symbol='y_fix')
y_1

Matrix([
[y_fix],
[  y_1]])

In [3]:
K = sp_utils.create_block_matrix()
K

Matrix([
[k1, k2],
[k3, k4]])

In [4]:
J = sp_utils.create_block_diagonal_matrix('J', 'j_1')
J

Matrix([
[J,   0],
[0, j_1]])

In [5]:
sp_utils.block_matrix_matrix_addition(K, J)

Matrix([
[J + k1,       k2],
[    k3, j_1 + k4]])

In [6]:
sp_utils.block_matrix_vector_product(K, y_1)

Matrix([
[k1*y_fix + k2*y_1],
[k3*y_fix + k4*y_1]])

In [7]:
sigma_2 = sp.Symbol('sigma_2')
sp_utils.scalar_block_matrix_product(sigma_2, K)

Matrix([
[sigma_2*k1, sigma_2*k2],
[sigma_2*k3, sigma_2*k4]])

In [8]:
sp_utils.block_matrix_matrix_product(K, J)


Matrix([
[k1*J, k2*j_1],
[k3*J, k4*j_1]])

In [9]:
sp_utils.block_matrix_matrix_product(K, K)

Matrix([
[k1**2 + k2*k3, k1*k2 + k2*k4],
[k3*k1 + k4*k3, k4**2 + k3*k2]])

In [10]:
sp_utils.block_matrix_matrix_product(J, K)

Matrix([
[  J*k1,   J*k2],
[j_1*k3, j_1*k4]])

In [11]:
sp_utils.block_matrix_matrix_product(J, J)

Matrix([
[J**2,      0],
[   0, j_1**2]])

In [12]:
sp_utils.inverse_block_matrix(K)

Matrix([
[k1**(-1) + 1/(-k3k1invk2 + k4)*k1**(-1)*k2*k3*k1**(-1), (-1/(-k3k1invk2 + k4))*k1**(-1)*k2],
[                    (-1/(-k3k1invk2 + k4))*k3*k1**(-1),    Matrix([[1/(-k3k1invk2 + k4)]])]])

In [13]:
sp_utils.transpose_vector(y_1)

Matrix([[y_fix.T, y_1.T]])

In [14]:
sp_utils.transpose_matrix(K)

Matrix([
[k1.T, k3.T],
[k2.T,   k4]])

In [15]:
y_1.blocks[0], y_1.blocks[1], y_1.blocks[0, 0], y_1.blocks[1, 0]

(y_fix, y_1, y_fix, y_1)

In [16]:
y_1.T.blocks[0], y_1.T.blocks[1]

(y_fix.T, y_1.T)

In [17]:
K.T

Matrix([
[k1.T, k3.T],
[k2.T, k4.T]])

In [18]:
sp_utils.block_matrix_vector_product(K, y_1)

Matrix([
[k1*y_fix + k2*y_1],
[k3*y_fix + k4*y_1]])

In [19]:
sp_utils.transposed_vector_matrix_product(
    sp_utils.transpose_vector(y_1),
    sp_utils.transpose_matrix(K)
)

Matrix([[y_1.T*k2.T + y_fix.T*k1.T, y_1.T*k4 + y_fix.T*k3.T]])

## Setup the main symbols

In [20]:
sigma_2 = sp.Symbol('sigma_2')
K = sp_utils.create_block_matrix()

# corruption 1 matrices
J_1 = sp_utils.create_block_diagonal_matrix('J', 'j_1')
m_1 = sp_utils.create_block_vector('m_1', V_fix_symbol='m_fix')
y_1 = sp_utils.create_block_vector('y_1', V_fix_symbol='y_fix')

# corruption 2 matrices
J_2 = sp_utils.create_block_diagonal_matrix('J', 'j_2')
m_2 = sp_utils.create_block_vector('m_2', V_fix_symbol='m_fix')
y_2 = sp_utils.create_block_vector('y_2', V_fix_symbol='y_fix')

# uncorrupted matrices
J_0 = sp_utils.create_block_diagonal_matrix('J', 'j_0')
m_0 = sp_utils.create_block_vector('m_0', V_fix_symbol='m_fix')
y_0 = sp_utils.create_block_vector('y_0', V_fix_symbol='y_fix')

In [21]:
# Getting mu_1 of the posterior distribution
def get_mu(K, J_1, m_1, y_1, sigma_2):
    K_tilde = sp_utils.block_matrix_matrix_addition(K, sp_utils.scalar_block_matrix_product(sigma_2, J_1))
    inv_K_tilde = sp_utils.inverse_block_matrix(K_tilde)
    adjusted_y = sp_utils.block_vector_vector_subtraction(y_1, m_1)
    mu = sp_utils.block_matrix_vector_product(
        sp_utils.block_matrix_matrix_product(K, inv_K_tilde),
        adjusted_y)
    return mu

mu_1 = get_mu(K, J_1, m_1, y_1, sigma_2)
mu_1

Matrix([
[((-1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k1*(sigma_2*J + k1)**(-1)*k2 + k2*Matrix([[1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])]]))*(-m_1 + y_1) + ((-1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k2*k3*(sigma_2*J + k1)**(-1) + k1*((sigma_2*J + k1)**(-1) + 1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])*(sigma_2*J + k1)**(-1)*k2*k3*(sigma_2*J + k1)**(-1)))*(-m_fix + y_fix)],
[((-1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k3*(sigma_2*J + k1)**(-1)*k2 + k4*Matrix([[1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])]]))*(-m_1 + y_1) + ((-1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k4*k3*(sigma_2*J + k1)**(-1) + k3*((sigma_2*J + k1)**(-1) + 1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])*(sigma_2*J + k1)**(-1)*k2*k3*(sigma_2*J + k1)**(-1)))*(-m_fix + y_fix)]])

In [22]:
def get_Cov(K, J, sigma_2):
    term_1 = sp_utils.block_matrix_matrix_addition(K, sp_utils.scalar_block_matrix_product(sigma_2, J))
    inv_term_1 = sp_utils.inverse_block_matrix(term_1)
    Cov = sp_utils.block_matrix_matrix_product(
        K,
        sp_utils.block_matrix_matrix_product(
            inv_term_1,
            sp_utils.scalar_block_matrix_product(sigma_2, J)
        )
    )
    return Cov
Cov_1 = get_Cov(K, J_1, sigma_2)
Cov_1

Matrix([
[sigma_2*k1*((sigma_2*J + k1)**(-1) + 1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])*(sigma_2*J + k1)**(-1)*k2*k3*(sigma_2*J + k1)**(-1))*J + (-sigma_2/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k2*k3*(sigma_2*J + k1)**(-1)*J, sigma_2*k2*Matrix([[1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])]])*j_1 + (-sigma_2/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k1*(sigma_2*J + k1)**(-1)*k2*j_1],
[sigma_2*k3*((sigma_2*J + k1)**(-1) + 1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])*(sigma_2*J + k1)**(-1)*k2*k3*(sigma_2*J + k1)**(-1))*J + (-sigma_2/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k4*k3*(sigma_2*J + k1)**(-1)*J, sigma_2*k4*Matrix([[1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])]])*j_1 + (-sigma_2/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k3*(sigma_2*J + k1)**(-1)*k2*j_1]])

In [23]:
def get_inv_Cov(K, J, sigma_2):
    inv_Cov = sp_utils.scalar_block_matrix_product(
        sigma_2**(-1),
        sp_utils.block_matrix_matrix_product(
            sp_utils.inverse_block_matrix(J_1),
            sp_utils.block_matrix_matrix_product(
                sp_utils.block_matrix_matrix_addition(
                    K,
                    sp_utils.scalar_block_matrix_product(sigma_2, J_1)
                ),
                sp_utils.inverse_block_matrix(K)
            )
        )
    )
    return inv_Cov
inv_Cov_1 = get_inv_Cov(K, J_1, sigma_2)
inv_Cov_1

Matrix([
[            1/sigma_2*J**(-1)*((sigma_2*J + k1)*(k1**(-1) + 1/(-k3k1invk2 + k4)*k1**(-1)*k2*k3*k1**(-1)) + 1/(k3k1invk2 - k4)*k2*k3*k1**(-1)),             1/sigma_2*J**(-1)*(k2*Matrix([[1/(-k3k1invk2 + k4)]]) + 1/(k3k1invk2 - k4)*(sigma_2*J + k1)*k1**(-1)*k2)],
[1/sigma_2*Matrix([[1/j_1]])*(k3*(k1**(-1) + 1/(-k3k1invk2 + k4)*k1**(-1)*k2*k3*k1**(-1)) + 1/(k3k1invk2 - k4)*(sigma_2*j_1 + k4)*k3*k1**(-1)), 1/sigma_2*Matrix([[1/j_1]])*((sigma_2*j_1 + k4)*Matrix([[1/(-k3k1invk2 + k4)]]) + 1/(k3k1invk2 - k4)*k3*k1**(-1)*k2)]])

## Computing the (2) term

In [24]:
# we are interested in (mu_1-mu)^T * inv_Cov * (mu_1-mu)
mu_1 = get_mu(K, J_1, m_1, y_1, sigma_2)
mu_0 = get_mu(K, J_0, m_0, y_0, sigma_2)
mu_diff_1 = sp_utils.block_vector_vector_subtraction(mu_1, mu_0)
mu_diff_transposed_1 = sp_utils.transpose_vector(mu_diff_1)

inv_Cov_1 = get_inv_Cov(K, J_1, sigma_2)

v_diff_1 = sp_utils.block_matrix_vector_product(
    inv_Cov_1,
    mu_diff_1
)

u1 = mu_diff_1.blocks[0]
u2 = mu_diff_1.blocks[1]

v1 = v_diff_1.blocks[0]
v2 = v_diff_1.blocks[1]

result_1 = u1 * v1 + u2 * v2
result_1


(-((-1/(-k3k1invk2 + sigma_2*j_0[0, 0] + k4[0, 0]))*k1*(sigma_2*J + k1)**(-1)*k2 + k2*Matrix([[1/(-k3k1invk2 + sigma_2*j_0[0, 0] + k4[0, 0])]]))*(-m_0 + y_0) - ((-1/(-k3k1invk2 + sigma_2*j_0[0, 0] + k4[0, 0]))*k2*k3*(sigma_2*J + k1)**(-1) + k1*((sigma_2*J + k1)**(-1) + 1/(-k3k1invk2 + sigma_2*j_0[0, 0] + k4[0, 0])*(sigma_2*J + k1)**(-1)*k2*k3*(sigma_2*J + k1)**(-1)))*(-m_fix + y_fix) + ((-1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k1*(sigma_2*J + k1)**(-1)*k2 + k2*Matrix([[1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])]]))*(-m_1 + y_1) + ((-1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0]))*k2*k3*(sigma_2*J + k1)**(-1) + k1*((sigma_2*J + k1)**(-1) + 1/(-k3k1invk2 + sigma_2*j_1[0, 0] + k4[0, 0])*(sigma_2*J + k1)**(-1)*k2*k3*(sigma_2*J + k1)**(-1)))*(-m_fix + y_fix))*(1/sigma_2*J**(-1)*((sigma_2*J + k1)*(k1**(-1) + 1/(-k3k1invk2 + k4)*k1**(-1)*k2*k3*k1**(-1)) + 1/(k3k1invk2 - k4)*k2*k3*k1**(-1))*(-((-1/(-k3k1invk2 + sigma_2*j_0[0, 0] + k4[0, 0]))*k1*(sigma_2*J + k1)**(-1)*k2 + k2*Matrix([

In [25]:
result_1.shape

(n, 1)